In [5]:
import sys
import os
import numpy as np
from dataset.mnist import load_mnist
from multi_layer_net import MultiLayerNet
from common.optimizer import Momentum
from common.layers import Affine, Relu, SoftmaxWithLoss, BatchNormalization

# 데이터 로드
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# 네트워크 생성
network = MultiLayerNet(input_size=784, hidden_size_list=[15, 15, 15], output_size=10,
                        activation='relu', weight_init_std='relu')

# 배치 정규화를 위한 레이어 추가
network.layers['BatchNorm1'] = BatchNormalization(100, beta=0.0)
network.layers['BatchNorm2'] = BatchNormalization(100, beta=0.0)
network.layers['BatchNorm3'] = BatchNormalization(100, beta=0.0)

# 하이퍼파라미터 설정
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100

# 학습률 범위 설정
learning_rates = np.arange(0.001, 0.02, 0.001)

# 은닉층 크기 범위 설정
hidden_sizes = np.arange(5, 16)

# 모멘텀 파라미터 설정
momentum = 0.9

train_loss_list = []  # 손실 함수 값 기록을 위한 리스트 추가
train_acc_list = []
test_acc_list = []

# 하이퍼파라미터 튜닝
for lr in learning_rates:
    for hidden_size in hidden_sizes:
        network.hidden_size_list = [hidden_size, hidden_size, hidden_size]
        optimizer = Momentum(lr, momentum)  # Momentum으로 변경

        train_loss = []  # 각 조합별 학습 손실을 기록하기 위한 리스트 추가
        train_acc = []
        test_acc = []

        for i in range(iters_num):
            batch_mask = np.random.choice(train_size, batch_size)
            x_batch = x_train[batch_mask]
            t_batch = t_train[batch_mask]

            # 순전파
            grads = network.gradient(x_batch, t_batch)
            optimizer.update(network.params, grads)

            # 정확도 및 손실 계산
            loss = network.loss(x_batch, t_batch)  # 학습 손실 계산
            train_loss.append(loss)  # 학습 손실을 리스트에 추가

            if i % 100 == 0:
                train_acc_val = network.accuracy(x_train, t_train)
                test_acc_val = network.accuracy(x_test, t_test)
                train_acc.append(train_acc_val)
                test_acc.append(test_acc_val)
                print(f"Iteration: {i}, Train Acc: {train_acc_val}, Test Acc: {test_acc_val}")

                # 94% 정확도 달성 시 종료
                if train_acc_val >= 0.99 and test_acc_val >= 0.97:
                    train_loss_list.append(train_loss)  # 손실 함수 값 기록 추가
                    train_acc_list.append(train_acc)
                    test_acc_list.append(test_acc)
                    print(f"Learning Rate: {lr}, Hidden Size: {hidden_size}")
                    break
        else:
            continue
        break
    else:
        continue
    break

# 손실 함수 값(정확도)을 그래프로 출력
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
for i, train_loss in enumerate(train_loss_list):
    # 정확도를 손실 함수 값으로 변환하여 그래프를 그립니다.
    train_loss = [1 - acc for acc in train_acc]
    iterations = range(len(train_loss))  # 반복 횟수
    plt.plot(iterations, train_loss, label=f"LR: {learning_rates[i]}, Hidden: {hidden_sizes[i]}")
plt.title('Train Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.ylim(0.0, 1.0)  # y 축 범위 설정
plt.legend()

plt.subplot(1, 2, 2)
for i, train_loss in enumerate(train_loss_list):
    # 정확도를 손실 함수 값으로 변환하여 그래프를 그립니다.
    train_loss = [1 - acc for acc in train_acc]
    iterations = range(len(train_loss))  # 반복 횟수
    plt.plot(iterations, train_loss, label=f"LR: {learning_rates[i]}, Hidden: {hidden_sizes[i]}")
plt.title('Test Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.ylim(0.0, 1.0)  # y 축 범위 설정
plt.legend()

plt.show()

ModuleNotFoundError: No module named 'multi_layer_net'

In [6]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class MultiLayerNet:
    """완전연결 다층 신경망

    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    weight_decay_lambda : 가중치 감소(L2 법칙)의 세기
    """
    def __init__(self, input_size, hidden_size_list, output_size,
                 activation='relu', weight_init_std='relu', weight_decay_lambda=0):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.weight_decay_lambda = weight_decay_lambda
        self.params = {}

        # 가중치 초기화
        self.__init_weight(weight_init_std)

        # 계층 생성
        activation_layer = {'sigmoid': Sigmoid, 'relu': Relu}
        self.layers = OrderedDict()
        for idx in range(1, self.hidden_layer_num+1):
            self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)],
                                                      self.params['b' + str(idx)])
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()

        idx = self.hidden_layer_num + 1
        self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)],
            self.params['b' + str(idx)])

        self.last_layer = SoftmaxWithLoss()

    def __init_weight(self, weight_init_std):
        """가중치 초기화
        
        Parameters
        ----------
        weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
            'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
            'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
        """
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he'):
                scale = np.sqrt(2.0 / all_size_list[idx - 1])  # ReLU를 사용할 때의 권장 초깃값
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])  # sigmoid를 사용할 때의 권장 초깃값
            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.
        
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블 
        
        Returns
        -------
        손실 함수의 값
        """
        y = self.predict(x)

        weight_decay = 0
        for idx in range(1, self.hidden_layer_num + 2):
            W = self.params['W' + str(idx)]
            weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2)

        return self.last_layer.forward(y, t) + weight_decay

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        """기울기를 구한다(수치 미분).
        
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        
        Returns
        -------
        각 층의 기울기를 담은 딕셔너리(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        for idx in range(1, self.hidden_layer_num+2):
            grads['W' + str(idx)] = numerical_gradient(loss_W, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_W, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        
        Returns
        -------
        각 층의 기울기를 담은 딕셔너리(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for idx in range(1, self.hidden_layer_num+2):
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW + self.weight_decay_lambda * self.layers['Affine' + str(idx)].W
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db

        return grads